# 4.9.1 Customer Data

## The script contains the following points:

### 01. Importing Libraries and Loading Files
### 02. Data Wrangling and Data Consistency Checking
### 03. Combining Customer Data to ords_prods
### 04. Exporting Merged Data

## 01. Importing Libraries and Loading Files

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Creating path varuable for loading files
path = r'C:\Users\widne\Documents\CareerFoundry Exercises\Data_Immersion\Achievement 4\03-2024 Instacart Basket Analysis'

#Loading customers.csv file as df_cust dataframe
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

## 02. Data Wrangling and Data Consistency Checking 

In [2]:
# Checking column names
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [3]:
# Dropping unnecessary columns
df_cust = df_cust.drop(columns=['First Name', 'Surnam', 'Gender', 'date_joined', 'fam_status'], index = 1)

In [4]:
df_cust.head()

,user_id,STATE,Age,n_dependants,income
0,26711,Missouri,48,3,165665
2,65803,Idaho,35,2,99568
3,125935,Iowa,40,0,42049
4,130797,Maryland,26,1,40374
5,133128,Kentucky,43,2,49643


In [5]:
# Renaming columns to a standard format
rename_dict = {'STATE' : 'state',
               'Age' : 'age',
               'n_dependants' : 'number_of_dependents',
               'fam_status' : 'marital_status'}
               
df_cust.rename(columns = rename_dict, inplace = True)

In [6]:
# Chekcing column renaming
df_cust.head()

,user_id,state,age,number_of_dependents,income
0,26711,Missouri,48,3,165665
2,65803,Idaho,35,2,99568
3,125935,Iowa,40,0,42049
4,130797,Maryland,26,1,40374
5,133128,Kentucky,43,2,49643


In [27]:
# Checking data types of columns
df_cust.describe()

,user_id,Age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [28]:
# Checking for mixed-type data
for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust[weird]) > 0:
        print(col)

C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\widne\AppData\Local

first_name


C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
C:\Users\widne\AppData\Local\Temp\ipykernel_19168\4091257574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)


In [8]:
# Since first name shows up as a mixed data type, the column is being changed to a string
df_cust['first_name'] = df_cust['first_name'].astype('str')

In [9]:
# Checking for missing values
df_cust.isnull().sum()

user_id                 0
first_name              0
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependents    0
marital_status          0
income                  0
dtype: int64

In [10]:
# Checking for duplicates
df_cust.shape

(206209, 10)

In [11]:
df_cust = df_cust.drop_duplicates()

In [12]:
df_cust.shape

(206209, 10)

## 03. Combining Customer Data to ords_prods

In [7]:
# Loading in ords_prods merged dataframe
ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

In [8]:
# Checking ords_prods dataframe
ords_prods.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,prices,_merge,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend,spender_flag,median_frequency,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,9.0,both,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,9.0,both,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,9.0,both,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,9.0,both,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,9.0,both,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [9]:
# Changing data type of user_id column of both the customer and prepared dataframe for merging
df_cust['user_id'] = df_cust['user_id'].astype('str')

In [10]:
ords_prods['user_id'] = ords_prods['user_id'].astype('str')

In [11]:
df_cust_ords_prods = ords_prods.merge(df_cust, on = 'user_id')

In [12]:
# Checking merge
df_cust_ords_prods.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,max_order,loyalty_flag,average_spend,spender_flag,median_frequency,frequency_flag,state,age,number_of_dependents,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Alabama,31,3,40423
1,2398795,1,2,3,7,15.0,False,196,1,1,...,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Alabama,31,3,40423
2,473747,1,3,3,12,21.0,False,196,1,1,...,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Alabama,31,3,40423
3,2254736,1,4,4,7,29.0,False,196,1,1,...,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Alabama,31,3,40423
4,431534,1,5,4,15,28.0,False,196,1,1,...,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Alabama,31,3,40423


In [13]:
df_cust_ords_prods.shape

(32404786, 27)

## 04. Exporting Merged Data

In [14]:
# Exporting merged dataframe as pickle
df_cust_ords_prods.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))